In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [2]:
# 读取数据
df=pd.read_pickle('data.pkl')

In [3]:
# 三折交叉
cvs=[32,31,30]
params = {
        'objective': 'tweedie',
        'tweedie_variance_power':1.6,
        'metric': 'mse',
        'num_leaves': 2**7-1,
        'reg_lambda': 50,
        'colsample_bytree': 0.6,
        'subsample': 0.6,
        'subsample_freq': 4,
        'learning_rate': 0.015,
        'n_estimators':2000,
        'seed': 1024,
        'n_jobs':-1,
        'silent': True,
        'verbose': -1,
    }

In [4]:
y_preds=[]
scores=[]
for cv in cvs:
    print('='*10+str(cv)+'='*10)
    train=df[df['week']<cv]
    val=df[df['week']==cv]
    test=df[df['week']==33]
    X_train=train.drop(columns=['weekly_sales'])
    y_train=train['weekly_sales']
    X_test=test.drop(columns=['weekly_sales'])
    y_test=test['weekly_sales']
    X_val=val.drop(columns=['weekly_sales'])
    y_val=val['weekly_sales']
    model=lgb.LGBMRegressor(**params)
    model.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_val,y_val)],eval_metric=['mse'],verbose=False,categorical_feature=['shop_id','item_id','item_category_id'],early_stopping_rounds=50)
    val_pred=model.predict(X_val)
    mse=mean_squared_error(y_val,val_pred)
    print(f'MSE: {mse}')
    scores.append(mse)
    y_pred=model.predict(X_test)
    y_preds.append(y_pred)
print(f'三折交叉的score{scores}')
print(f'三折交叉平均score{np.mean(scores)}')

==========32==========


D:\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
D:\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['item_category_id', 'item_id', 'shop_id']
  _log_warning('categorical_feature in Dataset is overridden.\n'


In [ ]:
y_pred=np.zeros_like(y_pred)
for t in y_preds:
    y_pred+=t*1/3

In [ ]:
sample_submit = pd.read_csv('./线下商店销量预测_数据集/sample_submit.csv')

In [ ]:
sample_submit['weekly_sales'] = y_pred
sample_submit['weekly_sales'] = sample_submit['weekly_sales'].apply(lambda x:x if x>0 else 0).values

In [ ]:
sample_submit

,shop_id,item_id,weekly_sales
0,0,0,0.522429
1,0,1,0.554448
2,0,2,2.275978
3,0,3,0.552268
4,0,4,1.039115
...,...,...,...
16731,31,518,0.441938
16732,31,519,0.452581
16733,31,520,0.479637
16734,31,521,0.547014


In [ ]:
sample_submit.to_csv('submit.csv', index=False)